#### 1. 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

import optuna
import optuna.visualization

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train = pd.read_csv('1. train.csv', encoding='utf-8')
train_x = train.drop(['EC'], axis=1)
train_y = train['EC']

test_x = pd.read_csv('2. test.csv', encoding = 'utf-8')

1-1. train

In [ ]:
train.columns

In [ ]:
train.index

In [ ]:
train.dtypes

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe(include='all')

In [ ]:
train['가중치가 적용된 열전도도 표준편차'].unique()

In [ ]:
train['가중치가 적용된 제1이온화 에너지 표준편차'].unique()

In [ ]:
train['전자 친화도 엔트로피'].unique()

In [ ]:
train['EC'].unique()

1-2. test

In [ ]:
test_x.info()

#### 2. EDA

2-1. 데이터 자료형과 통계량 확인

In [ ]:
train_x.info()

In [ ]:
# 각 변수들의 데이터 통계량을 확인합니다.
train_x.describe(include='all')

2-2. 데이터 시각화

In [ ]:
plt.figure(figsize=(12,12))
#sns.pairplot(train)
sns.pairplot(train.sample(3000)) # 오래 걸릴 시 수행
plt.show()

#범주형 변수 확인

In [ ]:
train_x.columns

In [ ]:
#범주형 변수는 distplot 생성 불가
fig, axes = plt.subplots(4,3, figsize=(20,20))

sns.histplot(x=train['가중치가 적용된 원자량의 범위'], kde=True, ax=axes[0][0]).set_title('가중치가 적용된 원자량의 범위')
sns.histplot(x=train['가중치가 적용된 열전도도 범위'], kde=True, ax=axes[0][1]).set_title('가중치가 적용된 열전도도 범위')
sns.histplot(x=train['평균 전자 친화도'], kde=True, ax=axes[0][2]).set_title('평균 전자 친화도')
sns.histplot(x=train['가중치가 적용된 밀도 범위'], kde=True, ax=axes[1][0]).set_title('가중치가 적용된 밀도 범위')
sns.histplot(x=train['가중치가 적용된 기하평균 제1이온화 에너지'], kde=True, ax=axes[1][1]).set_title('가중치가 적용된 기하평균 제1이온화 에너지')
sns.histplot(x=train['기하평균 원자 반지름'], kde=True, ax=axes[1][2]).set_title('기하평균 원자 반지름')
sns.histplot(x=train['기하평균 원자량'], kde=True, ax=axes[2][0]).set_title('기하평균 원자량')
sns.histplot(x=train['평균 열전도도'], kde=True, ax=axes[2][1]).set_title('평균 열전도도')
sns.histplot(x=train['밀도 표준편차'], kde=True, ax=axes[2][2]).set_title('밀도 표준편차')
sns.histplot(x=train['가중치가 적용된 평균 전자 친화도'], kde=True, ax=axes[3][0]).set_title('가중치가 적용된 평균 전자 친화도')
sns.histplot(x=train['가중치가 적용된 기하평균 전자 친화도'], kde=True, ax=axes[3][1]).set_title('가중치가 적용된 기하평균 전자 친화도')
sns.histplot(x=train['가중치가 적용된 원자 반지름 범위'], kde=True, ax=axes[3][2]).set_title('가중치가 적용된 원자 반지름 범위')

plt.show()

# 대부분의 변수는 정규분포를 따름

2-3. 범주형 변수 분석

In [ ]:
# 분석할 문자형 특성 리스트
categorical_features = ['가중치가 적용된 열전도도 표준편차', '가중치가 적용된 제1이온화 에너지 표준편차', '전자 친화도 엔트로피']

plt.figure(figsize=(18, 5))
for i, feature in enumerate(categorical_features):
    plt.subplot(1, len(categorical_features), i+1)
    sns.countplot(x=train_x[feature], hue=train_y)
    plt.title(f'{feature}별 클래스 분포')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.xticks(rotation=45)  # 라벨이 큰 경우 회전

plt.tight_layout()
plt.show()

print("\n--- 각 범주별 클래스 비율 ---")
for feature in categorical_features:
    print(f"\nFeature: {feature}")
    df_temp = pd.concat([train_x[feature], train_y], axis=1)
    df_temp.columns = [feature, 'target']

    # 각 범주 내에서 클래스 비율 계산
    ratio = (
        df_temp.groupby(feature)['target']
        .value_counts(normalize=True)
        .unstack(fill_value=0)
        .round(3)
    )
    print(ratio)


데이터가 유의미한 정보를 보이지 않는 범주형 변수 제거 (해당없음)

#### 3. 데이터 전처리

3-1. 결측치 처리

In [ ]:
# 데이터가 50% 이상 비어있는 컬럼은 삭제

drop_columns = []
for col in train_x.columns:
    if train_x[col].isnull().mean() > 0.5:
        drop_columns.append(col)

print(f"삭제된 컬럼: {drop_columns}")

In [ ]:
train_x.drop(drop_columns, axis=1, inplace=True)
train_x.info()

In [ ]:
test_x.drop(drop_columns, axis=1, inplace=True)
test_x.info()

수치형/범주형 컬럼 구분

In [ ]:
num_cols = train_x.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = train_x.select_dtypes(exclude=[np.number]).columns.tolist()

print("수치형 변수:", num_cols)
print("범주형 변수:", cat_cols)

결측치 처리

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

# (1) 수치형 변수 → KNNImputer
if len(num_cols) > 0:
    imputer_num = KNNImputer(n_neighbors=5)
    train_x[num_cols] = imputer_num.fit_transform(train_x[num_cols])
    test_x[num_cols] = imputer_num.transform(test_x[num_cols])
    print("수치형 변수 KNNImputer 완료")

# (2) 범주형 변수 → 최빈값(SimpleImputer)
if len(cat_cols) > 0:
    imputer_cat = SimpleImputer(strategy='most_frequent')
    train_x[cat_cols] = imputer_cat.fit_transform(train_x[cat_cols])
    test_x[cat_cols] = imputer_cat.transform(test_x[cat_cols])
    print("범주형 변수 최빈값 대체 완료")

    print(train_x.info())
    print(test_x.info())

3-2. 인코딩(One-Hot / Label) <br>
트리 기반 모델(RF, ET, XGB, LGBM) 사용 예정으로 Label Encoding으로 진행

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encoding(train, test, cols):

    for c in cols:
        # LabelEncoder 객체 생성
        le = LabelEncoder()

        # train 데이터의 해당 컬럼(c)으로 LabelEncoder 학습
        le.fit(train[c])

        # train 데이터의 해당 컬럼을 수치형으로 변환
        train[c] = le.transform(train[c])

        # test 데이터에만 존재하는 새로운 label이 있을 수 있으므로 예외처리
        # → test 데이터의 모든 고유값(label)을 순회하며, train에서 학습되지 않은 label이면 classes_에 수동 추가
        for label in np.unique(test[c]):
            if label not in le.classes_:
                le.classes_ = np.append(le.classes_, label)

        # 확장된 classes_를 사용해 test 데이터의 해당 컬럼을 수치형으로 변환
        test[c] = le.transform(test[c])

        # 진행 상황 출력
        print(f"컬럼 '{c}' Label Encoding 완료")

    return train, test

train_x_fin, test_x_fin = label_encoding(train_x.copy(), test_x.copy(), cat_cols)

print("전처리 완료: 결측치 대체 + 인코딩 완료")
print(train_x_fin.info())

In [ ]:
print(test_x_fin.info())

In [ ]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
le = LabelEncoder()

# 종속변수(train_y)에 대해 fit & transform
train_y_fin = le.fit_transform(train_y)

# 변환 결과 확인
print("변환 전 클래스 목록:", le.classes_)
print("변환 후 값 예시:", np.unique(train_y_fin))


3-3. 다중공선성과 상관관계 확인

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 수치형 컬럼만 대상으로 진행
feature = train_x_fin.select_dtypes(include=[np.number])

# 상수열(표준편차 0) 제거
feature = feature.loc[:, feature.std() > 0]

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(feature.values, i) for i in range(feature.shape[1])]
vif['features'] = feature.columns

# 다중공선성 높은 변수 리스트
picked = list(vif[vif['VIF Factor'] > 10].features)

print(picked)

vif

In [ ]:
temp = train_x_fin.corr()
#그림 사이즈 지정
fig, ax = plt.subplots(figsize=(12,12))

#삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래쪽 삼각형에 False)
mask = np.zeros_like(temp, dtype=bool)
mask[np.triu_indices_from(mask)] = True

#히트맵 그리기
sns.heatmap(temp,
            cmap='RdYlBu_r',
            annot=True if len(temp.columns) <= 20 else False, #실제값을 표시
            mask=mask,               #표시하지 않을 마스크 부분을 지정
            linewidths = .5,         #경계면 실선으로 구분하기
            cbar_kws={'shrink': .5}, #컬러비 크기 절반으로 줄이기
            vmin = -1, vmax = 1)     #컬러비 범위 -1 ~ 1

plt.show()

In [ ]:
def highlight_value(val):
    if abs(val) > 0.8:
        color = 'skyblue'
    elif abs(val) > 0.5:
        color = 'orange'
    else:
        color = ''
    return f"background-color: {color}"

display(temp.style.applymap(highlight_value))

3-4. 변수 선택<br>
train_x_fin(train data) 데이터셋에 다중공선성 존재하는 변수 없음.

3-5. 스케일링 <br>
트리 기반 모델(RF, ET, XGB, LGBM) 사용 예정으로 생략

#### 4. 분석 모델 설계

In [ ]:
# Logistic Regression은 확인용
lr = LogisticRegression()
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
xgb = XGBClassifier()
lgbm = LGBMClassifier()

4-1. 분석 모델 성능 평가

In [ ]:
train_x_splited, val_x, train_y_splited, val_y = train_test_split(train_x_fin, train_y_fin, test_size=0.2, random_state=10)

In [ ]:
# Logistic Regression
lr.fit(train_x_splited, train_y_splited)
lr_pred = lr.predict(val_x)
print("=== Logistic Regression ===")
print("Accuracy :", accuracy_score(val_y, lr_pred))
print("F1 Score :", f1_score(val_y, lr_pred, average='weighted'))
print(confusion_matrix(val_y, lr_pred))
print(classification_report(val_y, lr_pred))
print("\n")

# Random Forest
rf.fit(train_x_splited, train_y_splited)
rf_pred = rf.predict(val_x)
print("=== Random Forest ===")
print("Accuracy :", accuracy_score(val_y, rf_pred))
print("F1 Score :", f1_score(val_y, rf_pred, average='weighted'))
print(confusion_matrix(val_y, rf_pred))
print(classification_report(val_y, rf_pred))
print("\n")

# Extra Trees
et.fit(train_x_splited, train_y_splited)
et_pred = et.predict(val_x)
print("=== Extra Trees ===")
print("Accuracy :", accuracy_score(val_y, et_pred))
print("F1 Score :", f1_score(val_y, et_pred, average='weighted'))
print(confusion_matrix(val_y, et_pred))
print(classification_report(val_y, et_pred))
print("\n")

# XGBoost
xgb.fit(train_x_splited, train_y_splited)
xgb_pred = xgb.predict(val_x)
print("=== XGBoost ===")
print("Accuracy :", accuracy_score(val_y, xgb_pred))
print("F1 Score :", f1_score(val_y, xgb_pred, average='weighted'))
print(confusion_matrix(val_y, xgb_pred))
print(classification_report(val_y, xgb_pred))
print("\n")

# LightGBM
lgbm.fit(train_x_splited, train_y_splited)
lgbm_pred = lgbm.predict(val_x)
print("=== LightGBM ===")
print("Accuracy :", accuracy_score(val_y, lgbm_pred))
print("F1 Score :", f1_score(val_y, lgbm_pred, average='weighted'))
print(confusion_matrix(val_y, lgbm_pred))
print(classification_report(val_y, lgbm_pred))


4-2. GridSearchCV를 활용한 하이퍼파라미터 튜닝

In [ ]:
param_grid_rf = {
    'n_estimators': [200, 500, 800],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

In [ ]:
param_grid_et = {
    'n_estimators': [200, 500, 800],
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
param_grid_xgb = {
    'n_estimators': [300, 500, 700],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1]
}

In [ ]:
param_grid_lgbm = {
    'n_estimators': [300, 500, 700],
    'num_leaves': [31, 63, 127],
    'learning_rate': [0.01, 0.05, 0.1]
}

In [ ]:
# 데이터가 10000개 이상
from sklearn.model_selection import GridSearchCV

models = {
    'RandomForest': (rf, param_grid_rf),
    'ExtraTrees': (et, param_grid_et),
    'XGBoost': (xgb, param_grid_xgb),
    'LightGBM': (lgbm, param_grid_lgbm)
}

# 각 모델 내부 병렬(n_jobs)을 제한해 외부 GridSearch 병렬성과 충돌 방지
for m in [rf, et, xgb, lgbm]:
    m.set_params(n_jobs=1)

# GridSearchCV 실행
grid_search_results = {}

for name, (model, param_grid) in models.items():
    print(f"▶ {name} 하이퍼파라미터 탐색 중...")

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=3,                         # 데이터가 많을 때는 2~3로 줄여서 속도 개선
        scoring='f1_weighted',        # F1-score로 성능 측정
        n_jobs=-1,                    # 외부 병렬화 허용 (내부는 1로 제한)
        return_train_score=True,
        verbose=1
    )

    grid_search.fit(train_x_fin, train_y_fin)
    grid_search_results[name] = grid_search

    print(f"{name} 최적 파라미터: {grid_search.best_params_}")
    print(f"{name} 최고 F1 Score: {grid_search.best_score_:.4f}\n")


In [ ]:
results = []

for name, gs in grid_search_results.items():
    results.append({
        'Model': name,
        'Best F1 Score': round(gs.best_score_, 4), 
        'Best Params': gs.best_params_
    })

# F1-score 기준 내림차순 정렬 (성능 높을수록 좋음)
result_df = pd.DataFrame(results).sort_values('Best F1 Score', ascending=False)

display(result_df)


plt.figure(figsize=(8,5))
sns.barplot(x='Model', y='Best F1 Score', data=result_df, palette='viridis')
plt.title('Model Comparison by F1 Score')
plt.ylabel('F1 Score')
plt.xlabel('Model')
plt.ylim(0, 1)
plt.show()

In [ ]:
rf_best = grid_search_results['RandomForest'].best_estimator_
et_best = grid_search_results['ExtraTrees'].best_estimator_
xgb_best = grid_search_results['XGBoost'].best_estimator_
lgbm_best = grid_search_results['LightGBM'].best_estimator_

#### 5. 모델 학습

In [ ]:
# 모델 학습
rf_best.fit(train_x_fin, train_y_fin)
et_best.fit(train_x_fin, train_y_fin)
xgb_best.fit(train_x_fin, train_y_fin)
lgbm_best.fit(train_x_fin, train_y_fin)

#### 6. 예측값 생성

In [ ]:
# Validation set 분리 (Optuna 검증용)

train_x_splited, val_x_optuna, train_y_splited, val_y_optuna = train_test_split(
    train_x_fin, train_y_fin, test_size=0.2, random_state=42
)

# 각 모델의 검증 데이터 예측값 저장
val_probs = {
    'rf': rf_best.predict_proba(val_x_optuna),
    'et': et_best.predict_proba(val_x_optuna),
    'xgb': xgb_best.predict_proba(val_x_optuna),
    'lgbm': lgbm_best.predict_proba(val_x_optuna)
}


In [ ]:
# Optuna 목적함수 정의 (F1 최대화)

def objective(trial):
    # 가중치 제안
    w1 = trial.suggest_float('w_rf', 0.0, 1.0)
    w2 = trial.suggest_float('w_et', 0.0, 1.0)
    w3 = trial.suggest_float('w_xgb', 0.0, 1.0)
    w4 = trial.suggest_float('w_lgbm', 0.0, 1.0)

    # 가중치 합 1로 정규화
    weights = np.array([w1, w2, w3, w4])
    if np.sum(weights) == 0:
        return 0
    weights /= np.sum(weights)

    # 각 모델의 예측값(클래스 레이블)을 가중평균 → 확률이 없으므로 다수결 기반으로 근사
    blended_prob = (
    weights[0] * val_probs['rf'] +
    weights[1] * val_probs['et'] +
    weights[2] * val_probs['xgb'] +
    weights[3] * val_probs['lgbm']
    )

    # 클래스별 확률 중 가장 높은 값 선택
    blended_pred = np.argmax(blended_prob, axis=1)

    # F1-score 계산
    f1 = f1_score(val_y_optuna, blended_pred, average='weighted')
    return f1


In [ ]:
# Optuna 탐색 실행

# F1-score 최대화를 목표로 설정
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

# 최적 가중치 및 F1-score 출력
best_weights = study.best_params
print("Optuna Best F1 Score:", study.best_value)
print("Optuna Best Weights:", best_weights)

# 최적화 과정 시각화
optuna.visualization.plot_optimization_history(study).show()

# 최적 가중치 시각화 (막대그래프)
pd.Series(best_weights).plot(kind='bar', title='Optimal Ensemble Weights')
plt.ylabel('Weight Value')
plt.show()

In [ ]:
# 최적 가중치로 Test 데이터 예측

weights = np.array(list(best_weights.values()))
weights /= np.sum(weights)

test_probs = {
    'rf': rf_best.predict_proba(test_x_fin),
    'et': et_best.predict_proba(test_x_fin),
    'xgb': xgb_best.predict_proba(test_x_fin),
    'lgbm': lgbm_best.predict_proba(test_x_fin)
}

blended_test_prob = (
    weights[0] * test_probs['rf'] +
    weights[1] * test_probs['et'] +
    weights[2] * test_probs['xgb'] +
    weights[3] * test_probs['lgbm']
)

final_pred = np.argmax(blended_test_prob, axis=1)

print("최종 예측 완료")
print("예측 결과 shape:", final_pred.shape)
print("예측 클래스 분포:\n", np.unique(final_pred, return_counts=True))

#### 7. 제출 파일 생성

In [ ]:
submission = pd.read_csv('3. Sample_submission.csv', encoding = 'utf-8')
submission.head(5)

In [ ]:
submission['EC'] = final_pred
submission.head(5)

In [ ]:
submission['EC'] = np.where(submission['EC'] == 0, 'Complex', 
                            np.where(submission['EC'] == 1, 'Intermediate', 'Simple'))

submission.head(20)

In [ ]:
submission.to_csv('submit_final.csv', index=False)